# Розробка додатку для аналізу, рефакторингу та оптимізації коду FPGA-проектів

Мета роботи — створити базовий додаток, який за допомогою моделі `gpt-4o` здатний аналізувати, рефакторити та оптимізувати код на мовах **VHDL** або **Verilog**.

## Навчальні цілі
- Ознайомитися з бібліотекою `azure.ai.inference`.
- Навчитись використовувати LLM для технічного аналізу FPGA-коду.
- Налаштовувати параметри генерації (`temperature`, `max_tokens`).
- Провести тестування та оцінку якості результатів.


## 1️⃣ Розробка базового додатку для генерації тексту

Базовий застосунок виконує запит до моделі Azure OpenAI (`gpt-4o`) і отримує результат аналізу HDL-коду.


In [1]:
!pip install azure-ai-inference



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!pip install azure-ai-inference


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
!pip install azure-core


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

# 1️⃣ Ініціалізація клієнта
token = os.environ.get('GITHUB_TOKEN', 'your_token_here')  # або збережи токен у змінній середовища
endpoint = 'https://models.inference.ai.azure.com'
model_name = 'gpt-4o'

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)
print('✅ Клієнт ініціалізовано успішно')

✅ Клієнт ініціалізовано успішно


## 2️⃣ Додавання параметрів генерації (температура, токени)

Користувач може задавати параметри генерації, щоб контролювати стиль і довжину відповіді.


module adder(
    input [3:0] A, B,
    output [4:0] SUM
);
assign SUM = A + B;
endmodule


In [ ]:
print("=== FPGA Code Analyzer ===")
fpga_code = input("Введіть код FPGA (VHDL або Verilog):\n")
task_type = input("Оберіть дію (аналіз / оптимізація / рефакторинг): ")
temperature = float(input("Введіть температуру (0.0–1.0): "))
max_tokens = int(input("Введіть максимальну кількість токенів: "))
token = "ghu_токен_сюди"

prompt = (
    "Ти експерт з FPGA-розробки та HDL (VHDL/Verilog).\n"
    f"Проаналізуй наступний код. Завдання: {task_type}.\n\n"
    "1. Опиши, що робить цей модуль.\n"
    "2. Визнач потенційні проблеми з таймінгом, синтезом або лексичними помилками.\n"
    "3. Запропонуй оптимізації (зменшення затримок, площі, покращення читабельності).\n"
    "4. Покажи рефакторинг-код після оптимізації, якщо це доречно.\n\n"
    f"Код:\n```\n{fpga_code}\n```"
)

response = client.complete(
    messages=[
        {"role": "system", "content": "Ти експерт з FPGA і цифрової логіки."},
        {"role": "user", "content": prompt},
    ],
    model=model_name,
    temperature=temperature,
    max_tokens=max_tokens,
    top_p=1.0,
)

result = response.choices[0].message.content
print("\n=== Результат аналізу ===\n")
print(result)

# Збереження результату у файл
with open("fpga_analysis_report.txt", "w", encoding="utf-8") as f:
    f.write(result)

print("\n✅ Результат збережено у 'fpga_analysis_report.txt'")


=== FPGA Code Analyzer ===

=== Результат аналізу ===

Давайте проаналізуємо ваш код. Ось розгорнутий аналіз:

---

### 1. **Опис функціональності модуля**
Модуль `adder` реалізує 4-бітний суматор. Він приймає два 4-бітні вхідні числа `A` і `B` та обчислює їх суму, яка записується у 5-бітний вихідний сигнал `SUM`. 

- Вхідні порти:
  - `A`: 4-бітний вхід.
  - `B`: 4-бітний вхід.
- Вихідний порт:
  - `SUM`: 5-бітний вихід, який забезпечує можливість зберігання результату додавання з урахуванням переносу (carry).

Функціонально, цей модуль реалізує просте додавання двох чисел.

---

### 2. **Потенційні проблеми**
#### a) **Таймінгові проблеми**
- У цьому коді немає таймінгових проблем, оскільки тут немає жодних тактових сигналів (асинхронна логіка). Операція додавання виконується комбінаційно.

#### b) **Синтаксичні помилки**
- Код має **лексичну помилку** через неправильне форматування:
  - Пропущені коми між портами у визначенні модуля:
    ```verilog
    module adder( input [3:0] A, B

## 3️⃣ Тестування роботи додатку

| Тест № | Вхідний код | Завдання | Температура | Токени | Результат |
|:------:|--------------|-----------|--------------|---------|-----------|
| 1 | Простий 4-бітний суматор | Аналіз | 0.3 | 600 | Виявлено правильну логіку додавання. |
| 2 | D-тригер | Оптимізація | 0.7 | 800 | Запропоновано зменшення кількості елементів. |
| 3 | Лічильник з скиданням | Рефакторинг | 0.9 | 1000 | Покращено структуру always-блоків. |


## 4️⃣ Оцінка якості отриманих результатів

| Критерій | Оцінка (1–10) | Коментар |
|-----------|----------------|----------|
| Точність аналізу | 9 | Правильно розпізнає логіку HDL-коду. |
| Варіативність відповідей | 8 | Регулюється параметром `temperature`. |
| Час генерації | 10 | Відповідь за 1–3 секунди. |
| Якість рефакторингу | 8 | Оптимізує структуру, але потребує ручної перевірки. |


## 5️⃣ Пропозиції щодо покращення структури запитів

1. **Додати параметр вибору цільової платформи FPGA** (Xilinx, Intel, Lattice) для адаптації рекомендацій.
2. **Застосувати багатокроковий промпт (LangChain)** — спочатку аналіз, потім оптимізація, потім генерація нового коду.
3. **Додати оцінку ефективності рефакторингу** (кількість елементів логіки до/після).
4. **Розширити інтерфейс** — зробити GUI у Streamlit або Tkinter.
5. **Вивід результатів у PDF/HTML**, щоб створювати автоматичні звіти.


## ✅ Висновки

У ході роботи було створено консольний додаток, який аналізує, оптимізує та рефакторить код FPGA-проектів.
Розроблено гнучку систему параметрів генерації (`temperature`, `max_tokens`), проведено тестування
на різних прикладах і запропоновано покращення у структурі запитів.
